In [ ]:
import os
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pickle
import cv2

sys.path.append(str(Path(Path.cwd()).parent))
from src.utils import DATA_DIR, IMG, RHD

# Visualise

In [ ]:
set = 'training'

# load annotations of this set
anno_all = RHD.load_annot(DATA_DIR, set)

sample_id = 28140
anno = anno_all[sample_id]

print(len(anno_all))
for key, val in anno.items():
    print(key)

# load data
image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'color', '%.5d.png' % sample_id)))
mask = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'mask', '%.5d.png' % sample_id)))
depth = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'depth', '%.5d.png' % sample_id)))

# process rgb coded depth into float: top bits are stored in red, bottom in green channel
depth = RHD.depth_two_uint8_to_float(depth[:, :, 0], depth[:, :, 1])  # depth in meters from the camera

# get info from annotation dictionary
kp_coord_uv = anno['uv_vis'][:, :2]  # u, v coordinates of 42 hand keypoints, pixel
kp_visible = (anno['uv_vis'][:, 2] == 1)  # visibility of the keypoints, boolean
kp_coord_xyz = anno['xyz']  # x, y, z coordinates of the keypoints, in meters
camera_intrinsic_matrix = anno['K']  # matrix containing intrinsic parameters

# Project world coordinates into the camera frame
kp_coord_uv_proj = np.matmul(kp_coord_xyz, np.transpose(camera_intrinsic_matrix))
kp_coord_uv_proj = kp_coord_uv_proj[:, :2] / kp_coord_uv_proj[:, 2:]

# Visualize data
fig = plt.figure(figsize=(15, 15))
ax1 = fig.add_subplot('221')
ax2 = fig.add_subplot('222')
ax3 = fig.add_subplot('223')
ax4 = fig.add_subplot('224', projection='3d')
ax1.imshow(image)
ax1.plot(kp_coord_uv[kp_visible, 0], kp_coord_uv[kp_visible, 1], 'ro')
ax1.plot(kp_coord_uv_proj[kp_visible, 0], kp_coord_uv_proj[kp_visible, 1], 'gx')
ax2.imshow(depth)
ax3.imshow(mask)
ax4.scatter(kp_coord_xyz[kp_visible, 0], kp_coord_xyz[kp_visible, 1], kp_coord_xyz[kp_visible, 2])
ax4.view_init(azim=-90.0, elev=-90.0)  # aligns the 3d coord with the camera view
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')
plt.show()

# Workspace

In [ ]:
# PRODUCE the 21 subset using the segmentation masks
# We only deal with the more prominent hand for each frame and discard the second set of keypoints

# sample_id = 30 # not very visible
sample_id = 28139
anno = anno_all[sample_id]

keypoint_xyz = anno['xyz']
keypoint_vis = anno['uv_vis'][:, 2]
keypoint_uv = anno['uv_vis'][:, :2]
camera_intrinsic_matrix = anno['K']
image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'color', '%.5d.png' % sample_id)))
mask = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'mask', '%.5d.png' % sample_id)))

cond_l = np.logical_and(np.greater(mask, 1), np.less(mask, 18))
cond_r = np.greater(mask, 17)
hand_map_l = np.where(cond_l, 1, 0)
hand_map_r = np.where(cond_r, 1, 0)
num_px_left_hand = np.sum(hand_map_l)
num_px_right_hand = np.sum(hand_map_r)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(cond_l)
ax[1].imshow(cond_r)

kp_coord_xyz_left = keypoint_xyz[:21, :]
kp_coord_xyz_right = keypoint_xyz[-21:, :]

cond_left = np.logical_and(np.ones(kp_coord_xyz_left.shape), np.greater(num_px_left_hand, num_px_right_hand))
kp_coord_xyz21 = np.where(cond_left, kp_coord_xyz_left, kp_coord_xyz_right)

kp_coord_uv_proj = np.matmul(kp_coord_xyz21, np.transpose(camera_intrinsic_matrix))
kp_coord_uv_proj[:, :2] = kp_coord_uv_proj[:, :2]/kp_coord_uv_proj[:, 2:]

kp_coord_xy_proj = kp_coord_uv_proj.copy()
kp_coord_xy_proj[:, :2] = kp_coord_uv_proj[:, :2]*kp_coord_uv_proj[:, 2:]
kp_coord_xy_proj = np.matmul(kp_coord_xy_proj, np.transpose(np.linalg.inv(camera_intrinsic_matrix)))

print(np.allclose(kp_coord_xy_proj, kp_coord_xyz21))

keypoint_vis_left = keypoint_vis[:21]
keypoint_vis_right = keypoint_vis[-21:]
keypoint_vis21 = np.where(cond_left[:, 0], keypoint_vis_left, keypoint_vis_right).astype(bool)

keypoint_uv_left = keypoint_uv[:21, :]
keypoint_uv_right = keypoint_uv[-21:, :]
keypoint_uv21 = np.where(cond_left[:, :2], keypoint_uv_left, keypoint_uv_right)

bbox = RHD.get_bbox(kp_coord_uv_proj)

crop, uvd_gt_crop = RHD.crop_hand(image, kp_coord_uv_proj)

# Plot
fig, ax = plt.subplots()
ax.imshow(image)
# RHD.visualize_joints_2d(ax, kp_coord_uv_proj[RHD.REORDER_IDX])
RHD.draw_bbox(ax, bbox, (image.shape[1], image.shape[0]))
ax.plot(kp_coord_uv_proj[keypoint_vis21, 0], kp_coord_uv_proj[keypoint_vis21, 1], 'ro')

fig, ax = plt.subplots()
ax.imshow(crop)
# RHD.visualize_joints_2d(ax, uvd_gt_crop[RHD.REORDER_IDX])
ax.plot(uvd_gt_crop[keypoint_vis21, 0], uvd_gt_crop[keypoint_vis21, 1], 'ro')

In [ ]:
crop_rsz = IMG.resize_img(crop, (256, 256))
uvd_gt_crop_rsz = IMG.scale_points_WH(uvd_gt_crop, (crop.shape[1], crop.shape[0]), (256, 256))
fig, ax = plt.subplots(5, 5, figsize=(10, 10))
scoremap = RHD.create_multiple_gaussian_map(uvd_gt_crop_rsz, (256, 256), 25, keypoint_vis21)

for i in range(5):
    for j in range(5):
        k = np.ravel_multi_index((i, j), (5, 5))
        if k >= scoremap.shape[2]:
            break
        show_scoremap = cv2.cvtColor(scoremap[:, :, k], cv2.COLOR_GRAY2RGB)
        show_scoremap = cv2.normalize(show_scoremap, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
        show_scoremap[:, :, 0] = 0
        show_scoremap[:, :, 2] = 0
        show = 0.5*show_scoremap + 0.5*crop_rsz
    
        ax[i, j].imshow(show.astype('uint32'))
        ax[i, j].plot(uvd_gt_crop_rsz[keypoint_vis21, 0], uvd_gt_crop_rsz[keypoint_vis21, 1], 'r.')
#         RHD.visualize_joints_2d(ax[i, j], uvd_gt_crop_rsz[RHD.REORDER_IDX])

In [ ]:
revert = RHD.revert_hand_annot(image, kp_coord_uv_proj, uvd_gt_crop_rsz, (256, 256))

fig, ax = plt.subplots()
ax.imshow(image)
RHD.visualize_joints_2d(ax, revert[RHD.REORDER_IDX])

In [ ]:
pts = RHD.detect_keypoints_from_scoremap(scoremap)

fig, ax = plt.subplots()
ax.imshow(crop_rsz)
ax.plot(pts[:, 0], pts[:, 1], 'r.')
# RHD.visualize_joints_2d(ax, pts[RHD.REORDER_IDX])

In [ ]:
mask = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'mask', '%.5d.png' % sample_id)))

image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'color', '%.5d.png' % sample_id)))
tmp_hand_mask = np.greater(mask, 1)
bg_mask = np.logical_not(tmp_hand_mask)
hand_mask = np.stack([bg_mask, tmp_hand_mask], 2)

fig, ax = plt.subplots()
cur_mask = bg_mask
show_mask = np.stack([np.zeros(cur_mask.shape), np.zeros(cur_mask.shape), cur_mask], 2).astype('uint8')
show_img = IMG.scale_img_255(show_mask)
show_img = show_img*0.5 + image*0.5
show_img = IMG.scale_img_255(show_img)
ax.imshow(show_img)

In [ ]:
sample_id = 28140
anno                = anno_all[sample_id]
image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'color', '%.5d.png' % sample_id)))
mask = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', set, 'mask', '%.5d.png' % sample_id)))

mask                = np.asarray(mask)
K = anno['K']
keypoint_xyz        = anno['xyz']
keypoint_vis        = anno['uv_vis'][:, 2]
keypoint_uv         = anno['uv_vis'][:, :2]
# Find dominant hand
cond_l              = np.logical_and(np.greater(mask, 1),
                                     np.less(mask, 18))
cond_r              = np.greater(mask, 17)
hand_map_l          = np.where(cond_l, 1, 0)
hand_map_r          = np.where(cond_r, 1, 0)
num_px_left         = np.sum(hand_map_l)
num_px_right        = np.sum(hand_map_r)
kp_coord_xyz_left   = keypoint_xyz[:21, :]
kp_coord_xyz_right  = keypoint_xyz[-21:, :]
cond_left           = np.logical_and(np.ones(kp_coord_xyz_left.shape),
                                     np.greater(num_px_left,
                                                num_px_right))
kp_coord_xyz21      = np.where(cond_left,
                               kp_coord_xyz_left, kp_coord_xyz_right)
hand_side           = np.where(np.greater(num_px_left, num_px_right), 
                               0, # left hand 
                               1) # right hand
# Visibility
keypoint_vis_left   = keypoint_vis[:21]
keypoint_vis_right  = keypoint_vis[-21:]
keypoint_vis21      = np.where(cond_left[:, 0],
                          keypoint_vis_left,
                          keypoint_vis_right).astype('bool')

# UV
keypoint_uv_left    = keypoint_uv[:21, :]
keypoint_uv_right   = keypoint_uv[-21:, :]
uvd_gt              = np.where(cond_left[:, :2],
                               keypoint_uv_left,
                               keypoint_uv_right)
uvd_gt = RHD.xyz2uvd(kp_coord_xyz21, K)
# fig, ax = plt.subplots()
# ax.imshow(image)
# ax.plot(uvd_gt[keypoint_vis21, 0], uvd_gt[keypoint_vis21, 1], 'b.')

# Canonical
xyz_norm, kpt_scale             = RHD.norm_keypoint(kp_coord_xyz21)
xyz_gt_canon, inv_rot_mat       = RHD.canonical_transform(xyz_norm)
rot_mat                         = np.linalg.inv(inv_rot_mat)
xyz_gt_canon                    = np.squeeze(xyz_gt_canon)
if hand_side == 1:
    print('switch to')
    xyz_gt_canon[:, 2] = -xyz_gt_canon[:, 2]

fig, ax = plt.subplots()
RHD.visualize_joints_2d(ax, xyz_gt_canon[RHD.REORDER_IDX])
    

if hand_side == 1:
    print('switch back')
    xyz_gt_canon[:, 2] = -xyz_gt_canon[:, 2]

reform_xyz = np.matmul(xyz_gt_canon, rot_mat)*kpt_scale
reform_xyz += kp_coord_xyz21[0, :]
reform_uvd = RHD.xyz2uvd(reform_xyz, K)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image)
ax.plot(uvd_gt[keypoint_vis21, 0], uvd_gt[keypoint_vis21, 1], 'bo')
ax.plot(reform_uvd[keypoint_vis21, 0], reform_uvd[keypoint_vis21, 1], 'ro')

print(np.allclose(reform_uvd[:, :2], uvd_gt[:, :2]))
print(np.allclose(reform_xyz, kp_coord_xyz21))